In [1]:
%load_ext autoreload
%autoreload 2

import os, sys

os.environ['HF_ENDPOINT']='https://hf-mirror.com'

In [2]:
# import asyncio

from dotenv import dotenv_values
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.legacy.llms import OpenAILike as OpenAI
from qdrant_client import models
from tqdm.asyncio import tqdm

from pipeline.ingestion import build_pipeline, build_vector_store, read_data
from pipeline.qa import read_jsonl, save_answers
from pipeline.rag import QdrantRetriever, generation_with_knowledge_retrieval




config = dotenv_values(".env")
config = {'COLLECTION_NAME': 'aiops24_large',
          'VECTOR_SIZE': 768,
          'GLM_KEY': 'ba7d0d7930f621b8f9d2036deb55207a.En856p5uJTTXbxQv'
}
config

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'COLLECTION_NAME': 'aiops24_large',
 'VECTOR_SIZE': 768,
 'GLM_KEY': 'ba7d0d7930f621b8f9d2036deb55207a.En856p5uJTTXbxQv'}

In [3]:
# 初始化 LLM 嵌入模型 和 Reranker
llm = OpenAI(
    api_key=config["GLM_KEY"],
    model="glm-4",
    api_base="https://open.bigmodel.cn/api/paas/v4/",
    is_chat_model=True,
)
embeding = HuggingFaceEmbedding(
    # model_name="BAAI/bge-big-zh-v1.5",
    model_name="./models--BAAI--bge-base-zh-v1.5/snapshots/f03589ceff5aac7111bd60cfc7d497ca17ecac65",
    cache_folder="./",
    embed_batch_size=128,
)
Settings.embed_model = embeding

# 初始化 数据ingestion pipeline 和 vector store
client, vector_store = await build_vector_store(config, reindex=False)

collection_info = await client.get_collection(
    config["COLLECTION_NAME"] or "aiops24_large"
)

print(collection_info)

print(collection_info.points_count)



status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.DOT: 'Dot'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}
0


In [4]:
if collection_info.points_count == 0:
    data = read_data("data")
    pipeline = build_pipeline(llm, embeding, vector_store=vector_store)
    # 暂时停止实时索引
    await client.update_collection(
        collection_name=config["COLLECTION_NAME"] or "aiops24",
        optimizer_config=models.OptimizersConfigDiff(indexing_threshold=0),
    )
    await pipeline.arun(documents=data, show_progress=True, num_workers=1)
    # 恢复实时索引
    await client.update_collection(
        collection_name=config["COLLECTION_NAME"] or "aiops24",
        optimizer_config=models.OptimizersConfigDiff(indexing_threshold=20000),
    )
    print(len(data))


Generating embeddings: 100%|██████████| 244/244 [06:43<00:00,  1.65s/it]  


42139


In [7]:
from llama_index.core import QueryBundle
    

def find_cate(node):
    cates = set()
    for k, v in node.relationships.items():
        if 'file_path' in v.metadata:
            fp = v.metadata['file_path']
            cate = fp.replace('/mnt/workspace/aiops24-RAG-demo/demo/data/', '').split('/')[0]
            # print(cate)
            cates.add(cate)
    # print(cates)
    return list(cates)

hit = 0
nsame = 0
for query in queries:
    
    qd = query['document']
    query_str = query['query']
    query_bundle = QueryBundle(query_str=query_str)
    node_with_scores = await retriever.aretrieve(query_bundle)
    rds = set()
    for n in node_with_scores:
        ds = find_cate(n.node)
        for d in ds:
            rds.add(d)
    
    if qd in rds:
        hit += 1
    
    if qd != ','.join(rds):
        print(query, rds)
        nsame += 1
    
hit, nsame

{'id': 1, 'query': 'PCF与NRF对接时，一般需要配置哪些数据？', 'document': 'rcp'} {'umac', 'rcp'}
{'id': 2, 'query': 'ZXUN RCP部署成功后，各个虚机个数都是最少个数，是否可以一次性扩容完成？有哪些注意事项？', 'document': 'rcp'} {'umac', 'emsplus'}
{'id': 3, 'query': '如何排查PCF侧建立专载失败，发起Rx-ASR释放问题？', 'document': 'rcp'} {'umac', 'rcp'}
{'id': 4, 'query': 'Npcf_SMPolicyControl服务包含哪些操作？', 'document': 'rcp'} {'umac', 'rcp'}
{'id': 8, 'query': 'RCP在IaaS架构下有哪些GSU虚机？', 'document': 'rcp'} {'umac'}
{'id': 9, 'query': 'RCP如何将VoNR呼叫的Rx会话绑定到对应的N7会话？', 'document': 'rcp'} {'umac', 'rcp'}
{'id': 11, 'query': '发布虚机时最多可以为虚机分配几块网卡？', 'document': 'director'} {'umac', 'emsplus', 'director'}
{'id': 12, 'query': '虚拟资源包括哪些种类', 'document': 'director'} {'umac', 'director'}
{'id': 13, 'query': '物理机通用场景纳管1500台主机需要哪种部署规模、几个节点？', 'document': 'director'} {'emsplus', 'director'}
{'id': 16, 'query': '发布镜像时，镜像源支持哪几种方式？', 'document': 'director'} {'umac', 'director'}
{'id': 17, 'query': 'Director告警南向设置是在哪个界面上', 'document': 'director'} {'umac', 'director'}
{'id': 24, 'query': 'PCF哪

(91, 48)

In [12]:

from custom.template import QA_TEMPLATE, QA_TEMPLATE_2

print(QA_TEMPLATE_2)


retriever = QdrantRetriever(vector_store, embeding, similarity_top_k=5)

queries = read_jsonl("question.jsonl")

print(len(queries))

qid = [18]
# qid = [8, 9, 14, 17, 18, 20, 21, 23, 37, 72, 74, 92]

# 生成答案
print("Start generating answers...")

results = []
for query in queries:
    if query['id'] not in qid:
        continue
    print(query)
    result = await generation_with_knowledge_retrieval(
        query["query"], retriever, llm, qa_template=QA_TEMPLATE_2, 
        debug=True,
        only_retrieval=True,
    )
    results.append(result)
    


    上下文信息如下：
    ----------
    {context_str}
    ----------
    请你首选基于上下文信息而不是自己的知识，回答以下问题，可以分点作答，如果上下文信息没有相关知识，但你确实知道比较准确的答案，可以基于自己的知识进行回答，不要解释上下文中是否有，也不要复述上下文信息，尽量直接回答问题：
    {query_str}

    回答：    
103
Start generating answers...
{'id': 18, 'query': 'RCP包含哪些数据存储类服务？', 'document': 'rcp'}
retrieved:
[rcp]:# 服务处理能力配置
[rcp]:# 数据分析基础配置
[rcp]:# 专业维护
[rcp]:# 系统能力配置
[umac]:RCP
[NodeWithScore(node=TextNode(id_='12d2dc49-fa3b-47a9-8d52-98b2f9796598', embedding=None, metadata={'file_path': 'Npcf_PolicyManagement/zh-CN/tree/N_17870806.txt', 'file_name': 'N_17870806.txt', 'file_type': 'text/plain', 'file_size': 1261, 'creation_date': '2024-06-15', 'last_modified_date': '2024-03-12', 'document_title': '# 服务处理能力配置'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRel

In [10]:
for r in results:
    print(r.text)
    print('-------------')

RCP在IaaS架构下的GSU虚机为GSU-Common-M4C2规格，具体配置为2 vCPU/4G RAM/30G HD。其他详细信息未在上下文中提供。
-------------
RCP（Policy and Charging Rules Function）在将VoNR呼叫的Rx会话绑定到对应的N7会话的过程中，主要通过以下方式实现：

1. **会话授权时长配置**：在RCP策略管理GUI配置中，可以为N7会话控制策略设置“授权时长”参数值，这决定了会话的重授权时间点。

2. **接口类型和DNN类型的区分**：RCP可以区分会话所在的接口类型（N7）和DNN类型（IMS），针对不同的类型可以分别设置会话授权时长。

3. **生成随机偏移值**：在会话建立时，RCP在“会话授权时长最大随机偏移”范围内生成一个“随机偏移值”，这个值将会加到会话授权时长配置值上，形成实际的会话授权时长默认值。

4. **会话绑定**：通过上述配置，当Rx会话建立时，RCP会将这个会话与对应的N7会话绑定，确保两者使用相同的会话授权时长。

5. **会话重授权时间点传递**：RCP将计算出的会话重授权时间点通过N7接口的revalidationTime参数传递给SMF（Session Management Function），或者通过Gx接口的Revalidation-Time参数传递给PGW（Packet Data Network Gateway），从而实现Rx会话与N7会话的绑定。

请注意，上述流程是基于上下文信息中提供的配置和管理方法推断的，实际操作中可能还需要考虑其他因素和协议细节。
-------------
在EPS回落流程中，根据提供的上下文信息，PCF向SBC上报用户位置的次数及使用的事件没有直接提及。但是，根据上下文信息中PCF支持下发给SMF的默认事件列表中包括“用户位置信息改变”，可以推测在EPS回落流程中，如果用户位置发生变化，PCF可能会通过这个事件至少上报一次用户位置信息给SBC。

具体次数和事件可能如下：
1. 当用户位置发生变化时，PCF可能会使用“用户位置信息改变”事件向SBC上报一次用户位置。

请注意，这只是一个基于上下文信息的推测，实际情况可能更复杂，并可能涉及其他事件或上报次数。
-------------
Director告警

In [ ]:

# 生成答案
print("Start generating answers...")

results = []
for query in tqdm(queries, total=len(queries)):
    result = await generation_with_knowledge_retrieval(
        query["query"], retriever, llm, qa_template=QA_TEMPLATE_2,
    )
    results.append(result)

# 处理结果
save_answers(queries, results, "submit_result_large.jsonl")

Start generating answers...


 17%|█▋        | 18/103 [02:00<06:03,  4.28s/it]